In [ ]:
# Install TensorFlow and DeepLab
!pip install tensorflow==2.6.0
!pip install deeplab

In [ ]:
# Download Cityscapes dataset
!wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=YOUR_USERNAME&password=YOUR_PASSWORD' https://www.cityscapes-dataset.com/login/
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=1
!unzip leftImg8bit_trainvaltest.zip -d cityscapes

In [ ]:
!python deeplab/datasets/remove_gt_colormap.py --input_folder=cityscapes/gtFine --output_folder=cityscapes/gtFine_labelIds
!python deeplab/datasets/create_cityscapes_tfrecord.py --image_folder=cityscapes/leftImg8bit --semantic_segmentation_folder=cityscapes/gtFine_labelIds --list_folder=cityscapes --output_dir=cityscapes/tfrecord\

In [ ]:
# model
from deeplab import common
from deeplab import model
from deeplab import input_preprocess

config = common.parse_from_config_file('deeplab/configs/cityscapes.yaml')
model = model.DeepLabModel(config)

train_dataset = input_preprocess.load('cityscapes/tfrecord/train', config, is_training=True)
val_dataset = input_preprocess.load('cityscapes/tfrecord/val', config, is_training=False)

In [ ]:
# loss function
from deeplab import losses
from deeplab import utils

def compute_loss(model, images, labels):
    logits = model(images, training=True)
    return losses.cross_entropy(logits, labels, config)

from deeplab import lr_schedule
from deeplab import optimizer

learning_rate_fn = lr_schedule.create_learning_rate_schedule(config)
optimizer_fn = optimizer.create_optimizer(config)

In [ ]:
# training 
from deeplab import training

train_summary_writer = tf.summary.create_file_writer('logs/train')
val_summary_writer = tf.summary.create_file_writer('logs/val')

@tf.function
def train_step(model, images, labels, optimizer):
    with tf.GradientTape() as tape:
        loss = compute_loss(model, images, labels)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

@tf.function
def val_step(model, images, labels):
    logits = model(images, training=False)
    loss = losses.cross_entropy(logits, labels, config)
    return loss

for epoch in range(config.train_config.num_epochs):
    with train_summary_writer.as_default():
        for step, (images, labels) in enumerate(train_dataset):
            loss = train_step(model, images, labels, optimizer_fn(learning_rate_fn(epoch)))
            tf.summary.scalar('loss', loss, step=step)
    with val_summary_writer.as_default():
        for images, labels in val_dataset:
            loss = val_step(model, images, labels)
            tf.summary.scalar('val_loss', loss, step=epoch)

In [ ]:
# evaluation
from deeplab import metrics

def evaluate(model, dataset):
    metric = metrics.MeanIoU(num_classes=config.num_classes)
    for images, labels in dataset:
        logits = model(images, training=False)
        predictions = tf.argmax(logits, axis=-1)
        metric.update_state(labels, predictions)
    miou = metric.result()
    return miou.numpy()

latest_checkpoint = tf.train.latest_checkpoint('checkpoints')
model.load_weights(latest_checkpoint)

val_dataset = input_preprocess.load('cityscapes/tfrecord/val', config, is_training=False)

miou = evaluate(model, val_dataset)
print('Validation mIoU:', miou)
